# Evaluation Of A Manufacturing Process

Manufacturing processes are akin to assembling a complex puzzle. Each product is meticulously constructed step by step, and maintaining a vigilant oversight over the entire process is paramount.

In this project, I will be providing support to a team striving to enhance their monitoring and control methodologies within a manufacturing environment. The objective is to introduce a systematic approach known as statistical process control (SPC). SPC is a well-established methodology that utilizes data-driven insights to ascertain the efficacy of a process. Adjustments are made solely when measurements deviate from predetermined acceptable parameters.

These parameters are delineated by an upper control limit (UCL) and a lower control limit (LCL), calculated using the following formulas:

$ucl = avg\_height + 3 * \frac{stddev\_height}{\sqrt{5}}$

$lcl = avg\_height - 3 * \frac{stddev\_height}{\sqrt{5}}$

The UCL denotes the maximum acceptable height for parts, whereas the LCL denotes the minimum acceptable height. Ideally, parts should conform to these established limits.

By leveraging SQL window functions and nested queries, I will analyzed historical manufacturing data to delineate this acceptable range. Moreover, I will identify any deviations from this range within the process, thereby necessitating corrective actions. This approach ensures the consistent operation of the manufacturing process, facilitating the production of high-quality products.

## The data
The data is available in the `manufacturing_parts` table which has the following fields:
- `item_no`: the item number
- `length`: the length of the item made
- `width`: the width of the item made
- `height`: the height of the item made
- `operator`: the operating machine

## Procedure:
First I will create an alert that flags whether the height of a product is within the control limits for each operator using the formulas provided in the notebook. The final query should return the following fields: operator, row_number, height, avg_height, stddev_height, ucl, lcl, alert, and be ordered by the the item_no. I will use a window function of length 5 considering rows before and including the current row. Note that I will remove any incomplete window rows in the final query output.

In [3]:
-- Calculating Moving Average and Moving Standard Deviation

SELECT
    operator,
    item_no,
    AVG(height) OVER window_name AS avg_height,
    STDDEV(height) OVER window_name AS stddev_height
FROM 
    manufacturing_parts
WINDOW window_name AS (
    PARTITION BY operator
    ORDER BY item_no
    ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
);

,operator,item_no,avg_height,stddev_height
0,Op-1,1,19.690000,NaN
1,Op-1,2,19.660000,0.042426
2,Op-1,3,20.276667,1.068519
3,Op-1,4,19.857500,1.209941
4,Op-1,5,19.778000,1.062812
...,...,...,...,...
495,Op-9,236,18.904000,0.203052
496,Op-9,237,18.864000,0.250260
497,Op-9,238,19.266000,0.952276
498,Op-9,239,19.678000,1.291112


In [4]:
-- Calculating Upper and Lower Control Limits

SELECT
       a.*,
	   a.avg_height + 3*a.stddev_height/SQRT(5) AS ucl,
	   a.avg_height - 3*a.stddev_height/SQRT(5) AS lcl 
FROM (
	SELECT
    operator,
    item_no,
    AVG(height) OVER window_name AS avg_height,
    STDDEV(height) OVER window_name AS stddev_height
FROM 
    manufacturing_parts
WINDOW window_name AS (
    PARTITION BY operator
    ORDER BY item_no
    ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
 )
) AS a;

,operator,item_no,avg_height,stddev_height,ucl,lcl
0,Op-1,1,19.690000,NaN,NaN,NaN
1,Op-1,2,19.660000,0.042426,19.716921,19.603079
2,Op-1,3,20.276667,1.068519,21.710236,18.843098
3,Op-1,4,19.857500,1.209941,21.480807,18.234193
4,Op-1,5,19.778000,1.062812,21.203912,18.352088
...,...,...,...,...,...,...
495,Op-9,236,18.904000,0.203052,19.176422,18.631578
496,Op-9,237,18.864000,0.250260,19.199759,18.528241
497,Op-9,238,19.266000,0.952276,20.543613,17.988387
498,Op-9,239,19.678000,1.291112,21.410208,17.945792


In [5]:
-- Flag whether the height of a product is within the control limits

SELECT
	b.*,
	CASE
		WHEN 
			b.height NOT BETWEEN b.lcl AND b.ucl
		THEN TRUE
		ELSE FALSE
	END as alert
FROM (
	SELECT
		a.*, 
		a.avg_height + 3*a.stddev_height/SQRT(5) AS ucl, 
		a.avg_height - 3*a.stddev_height/SQRT(5) AS lcl  
	FROM (
		SELECT 
			operator,
			ROW_NUMBER() OVER w , 
			height, 
			AVG(height) OVER w AS avg_height, 
			STDDEV(height) OVER w AS stddev_height
		FROM manufacturing_parts 
		WINDOW w AS (
			PARTITION BY operator 
			ORDER BY item_no 
			ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
		)
	) AS a
	WHERE a.row_number >= 5
) AS b

,operator,row_number,height,avg_height,stddev_height,ucl,lcl,alert
0,Op-1,5,19.46,19.778,1.062812,21.203912,18.352088,False
1,Op-1,6,20.36,19.912,1.090812,21.375477,18.448523,False
2,Op-1,7,20.22,20.030,1.084574,21.485108,18.574892,False
3,Op-1,8,21.03,19.934,0.931225,21.183369,18.684631,False
4,Op-1,9,19.78,20.170,0.598832,20.973418,19.366582,False
...,...,...,...,...,...,...,...,...
415,Op-9,31,19.01,18.904,0.203052,19.176422,18.631578,False
416,Op-9,32,18.57,18.864,0.250260,19.199759,18.528241,False
417,Op-9,33,20.91,19.266,0.952276,20.543613,17.988387,True
418,Op-9,34,21.24,19.678,1.291112,21.410208,17.945792,False
